# 05-02 - Agent Group Chat

## Overview 


## Environment

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.38.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.38.0"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using System.ComponentModel;
using DotNetEnv;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("SK_OPENAI_APIKEY");
string endpoint = Env.GetString("SK_OPENAI_ENDPOINT");
string chatCompletionDeployment = Env.GetString("SK_OPENAI_CHATCOMPLETION_DEPLOYMENT_DEFAULT");


Console.WriteLine($"Configuration loaded...");

Installed Packages DotNetEnv, 3.1.1 Microsoft.SemanticKernel, 1.38.0 Microsoft.SemanticKernel.Agents.Core, 1.38.0-alpha Microsoft.SemanticKernel.Connectors.OpenAI, 1.38.0

Configuration loaded...


## Plugin Definition

In [ ]:
public class RetrieveWinner
{
    [KernelFunction("get_sport_event_winner")]
    //Description is just necessary if function name is not self-explanatory
    [Description("Get the winner of a sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventWinner(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the winner of the sport event.
        return "Munich Ice Tiger Flying Dolphins";
    }
}

public class RetrieveScore
{
    [KernelFunction("get_sport_event_score")]
    //Description is just necessary if function name is cryptic and not self-explanatory
    [Description("Get the score of a specific sport event. The event is identified by the sport event name and the year.")]
    public string GetSportEventScore(string sportEventName = "", string sportEventYear = "")
    {
        // Implement the logic to get the result of the sport event.
        return "24:1";
    }
}

Console.WriteLine($"Plugins defined...")

Plugins defined...


## Agent Definition


In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001

//Define RetrieveWinner Agent
IKernelBuilder retrieveWinnerKernelBuilder = Kernel.CreateBuilder();
retrieveWinnerKernelBuilder.AddAzureOpenAIChatCompletion(
    apiKey: apiKey, 
    endpoint: endpoint, 
    deploymentName: chatCompletionDeployment
);
retrieveWinnerKernelBuilder.Plugins.AddFromType<RetrieveWinner>("RetrieveWinner");
Kernel retrieveWinnerKernel = retrieveWinnerKernelBuilder.Build();

string agentName = "RetrieveWinner";
string agentInstruction = "You can retrieve the winner of a sport event";

ChatCompletionAgent retrieveWinnerAgent = new ChatCompletionAgent()
{
    Name = agentName,
    Instructions = agentInstruction,
    Kernel = retrieveWinnerKernel,
    Arguments = new KernelArguments(
        new OpenAIPromptExecutionSettings() { 
            FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
        }
    )
};

//Define retrieveScore Agent
IKernelBuilder retrieveScoreKernelBuilder = Kernel.CreateBuilder();
retrieveScoreKernelBuilder.AddAzureOpenAIChatCompletion(
    apiKey: apiKey, 
    endpoint: endpoint, 
    deploymentName: chatCompletionDeployment
);
retrieveScoreKernelBuilder.Plugins.AddFromType<RetrieveScore>("RetrieveScore");
Kernel retrieveScoreKernel = retrieveScoreKernelBuilder.Build();

agentName = "RetrieveScore";
agentInstruction = "You can retrieve the final score of a sport event";

ChatCompletionAgent retrieveScoreAgent = new ChatCompletionAgent()
{
    Name = agentName,
    Instructions = agentInstruction,
    Kernel = retrieveScoreKernel,
    Arguments = new KernelArguments(
        new OpenAIPromptExecutionSettings() { 
            FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
        }
    )
};

Console.WriteLine($"retrieveWinner Agent created...");
Console.WriteLine($"retrieveScore Agent created...");

retrieveWinner Agent created...
retrieveScore Agent created...


## Create Group Chat

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001

AgentGroupChat agentGroupChat = new AgentGroupChat(retrieveWinnerAgent, retrieveScoreAgent);
agentGroupChat.ExecutionSettings = new AgentGroupChatSettings(){
    TerminationStrategy = { MaximumIterations = 2 }
};

Console.WriteLine($"AgentGroupChat created...");

AgentGroupChat created...


## Execute Group Chat

In [ ]:
#pragma warning disable SKEXP0110

string prompt = "Who won the Super Sports Championship 2025. Return name of the winning team and the score.";

agentGroupChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, prompt));

string response = "";
await foreach (ChatMessageContent chatMessageContent in agentGroupChat.InvokeAsync())
{
    response = String.Concat(response, chatMessageContent.Content, "\n");
    Console.WriteLine(chatMessageContent.Content);
}

The winner of the Super Sports Championship 2025 was the Munich Ice Tiger Flying Dolphins. Unfortunately, the score is not available in the retrieved information.
The Munich Ice Tiger Flying Dolphins won the Super Sports Championship 2025 with a score of 24:1.
